In [1]:
from keras.models import *
from keras.layers import *
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from tensorflow import keras
from keras import optimizers
import pandas as pd
import numpy as np
import tensorflow as tf


Using TensorFlow backend.


In [2]:
# color 전처리 파일 load
train_color = pd.read_csv('./data/dacon_g/train_colorindex.csv', index_col='id')
test_color = pd.read_csv('./data/dacon_g/test_colorindex.csv', index_col='id')
sample_submission = pd.read_csv('./data/dacon_g/sample_submission.csv', index_col='id')

In [11]:
train_color_y

id
0          8
1          8
2          8
3          8
4         10
          ..
199986     8
199987     6
199988    10
199989     6
199990     8
Name: type_num, Length: 199991, dtype: int64

In [7]:
# color 전처리
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train_color['type_num'] = train_color['type'].apply(lambda x: to_number(x, column_number))

In [8]:
train_color_x = train_color.drop(columns=['Unnamed: 0','type', 'fiberID','type_num'], axis=1)
train_color_y = train_color['type_num']
test_color_x = test_color.drop(columns=['fiberID','Unnamed: 0'], axis=1)

In [ ]:
# 교체할 것
# train_x = train.drop(columns=['type', 'fiberID','type_num'], axis=1)
# train_y = train['type_num']
# test_x = test.drop(columns=['fiberID'], axis=1)

In [19]:
test_color_x

,psfMag_g,fiberMag_g,petroMag_g,modelMag_g,fiber_u-g,fiber_g-r,fiber_r-i,fiber_i-z,psf_u-g,psf_g-r,psf_r-i,psf_i-z,petro_u-g,petro_g-r,petro_r-i,petro_i-z,model_u-g,model_g-r,model_r-i,model_i-z
id,,,,,,,,,,,,,,,,,,,,
199991,22.508963,23.167848,22.796239,22.499435,1.886041,1.831947,2.500043,1.401474,1.308437,1.527857,2.463790,1.441237,-0.549543,1.600924,2.610829,1.430202,2.892099,1.487517,2.512577,1.407867
199992,21.937111,22.186000,21.837511,21.853442,0.312564,1.567121,0.317676,0.271126,0.869872,1.601341,0.335258,0.473143,-0.107680,1.641382,0.228925,0.283532,0.621896,1.680273,0.376412,0.229385
199993,19.235669,19.439533,18.710223,18.653338,1.766013,1.095100,0.434743,0.371526,1.788581,0.931608,0.495453,0.428495,2.012406,1.098371,0.453332,0.314533,1.925976,1.091230,0.441579,0.411782
199994,20.286261,20.611498,20.385262,20.280943,0.364634,0.044237,0.087944,0.201846,0.217164,0.089057,0.034785,0.102587,-0.055993,0.256105,-0.077418,-0.005768,0.198937,0.130443,-0.055721,0.113311
199995,22.668237,22.935289,22.957496,22.857290,2.746572,1.292832,2.017531,1.033176,1.576614,1.428904,1.954555,1.048838,-0.649198,1.672463,1.985913,0.991593,2.632069,1.665429,1.953898,0.957596
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
209995,19.542406,19.827172,19.549257,19.536518,0.266136,0.113140,0.027172,-0.122836,0.222629,0.103120,0.081328,0.000652,0.333472,0.091692,0.154203,-0.180028,0.255663,0.104056,0.056017,0.021469
209996,19.609379,19.928523,19.635609,19.604800,0.391391,0.232261,-0.067509,0.356659,0.350651,0.283430,-0.065341,0.132427,0.449845,0.253686,-0.078593,0.069652,0.461752,0.270688,-0.067127,0.241828
209997,20.276182,20.551190,20.315201,20.255485,0.703264,0.185390,0.164928,0.270447,0.945717,0.185407,0.163567,0.160476,0.682819,0.078475,0.177925,0.070438,0.790016,0.113936,0.235535,-0.047918


In [12]:
scaler = RobustScaler()

In [13]:
_mat = scaler.fit_transform(train_color_x)
train_x = pd.DataFrame(_mat, columns=train_color_x.columns, index=train_color_x.index)


In [15]:
train_y = train_color_y

In [20]:
%%time
model = Sequential()
model.add(Dense(256, kernel_initializer='random_uniform', input_shape=(20,), activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(512, kernel_initializer='random_uniform',  activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(256, kernel_initializer='random_uniform', activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(512, kernel_initializer='random_uniform', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, kernel_initializer='random_uniform', activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(512, kernel_initializer='random_uniform', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, kernel_initializer='random_uniform', activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(512, kernel_initializer='random_uniform', activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(256, kernel_initializer='random_uniform', activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(19, kernel_initializer='random_uniform', activation='softmax'))

# 3. 모델 학습과정 설정하기
# Adadelta(learning_rate=1.0, rho=0.95)
Adamax = optimizers.Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999)
Adam = optimizers.Adam(learning_rate=0.0005,amsgrad=True)
model.compile(optimizer=Adamax, loss='sparse_categorical_crossentropy', metrics=["accuracy"])
 
# 4. 모델 학습시키기
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=1, mode='auto')
mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)

hist = model.fit(train_x, train_y, epochs=300, batch_size = 256*2, callbacks=[early_stopping,mc], validation_split=0.2, verbose=2)

Train on 159992 samples, validate on 39999 samples
Epoch 1/300
 - 19s - loss: 2.6074 - accuracy: 0.2865 - val_loss: 1.7047 - val_accuracy: 0.4664
Epoch 2/300
 - 19s - loss: 1.5117 - accuracy: 0.5182 - val_loss: 1.1425 - val_accuracy: 0.5925
Epoch 3/300
 - 19s - loss: 1.1279 - accuracy: 0.6126 - val_loss: 0.8612 - val_accuracy: 0.7117
Epoch 4/300
 - 19s - loss: 0.9132 - accuracy: 0.6964 - val_loss: 0.7029 - val_accuracy: 0.7761
Epoch 5/300
 - 19s - loss: 0.7954 - accuracy: 0.7414 - val_loss: 0.6245 - val_accuracy: 0.7983
Epoch 6/300
 - 19s - loss: 0.7291 - accuracy: 0.7650 - val_loss: 0.5939 - val_accuracy: 0.8106
Epoch 7/300
 - 19s - loss: 0.6753 - accuracy: 0.7819 - val_loss: 0.5578 - val_accuracy: 0.8238
Epoch 8/300
 - 19s - loss: 0.6514 - accuracy: 0.7911 - val_loss: 0.5413 - val_accuracy: 0.8286
Epoch 9/300
 - 19s - loss: 0.6196 - accuracy: 0.8007 - val_loss: 0.5221 - val_accuracy: 0.8313
Epoch 10/300
 - 19s - loss: 0.5983 - accuracy: 0.8068 - val_loss: 0.5161 - val_accuracy: 0.831

Epoch 86/300
 - 19s - loss: 0.3700 - accuracy: 0.8698 - val_loss: 0.3677 - val_accuracy: 0.8720
Epoch 87/300
 - 19s - loss: 0.3708 - accuracy: 0.8705 - val_loss: 0.3656 - val_accuracy: 0.8739
Epoch 88/300
 - 19s - loss: 0.3705 - accuracy: 0.8699 - val_loss: 0.3662 - val_accuracy: 0.8711
Epoch 89/300
 - 19s - loss: 0.3790 - accuracy: 0.8694 - val_loss: 0.3673 - val_accuracy: 0.8732
Epoch 90/300
 - 19s - loss: 0.3687 - accuracy: 0.8702 - val_loss: 0.3675 - val_accuracy: 0.8704
Epoch 91/300
 - 26s - loss: 0.3691 - accuracy: 0.8706 - val_loss: 0.3649 - val_accuracy: 0.8736
Epoch 92/300
 - 24s - loss: 0.3679 - accuracy: 0.8714 - val_loss: 0.3613 - val_accuracy: 0.8735
Epoch 93/300
 - 29s - loss: 0.3673 - accuracy: 0.8714 - val_loss: 0.3653 - val_accuracy: 0.8719
Epoch 94/300
 - 19s - loss: 0.3682 - accuracy: 0.8707 - val_loss: 0.3631 - val_accuracy: 0.8738
Epoch 95/300
 - 19s - loss: 0.3673 - accuracy: 0.8711 - val_loss: 0.3636 - val_accuracy: 0.8731
Epoch 96/300
 - 19s - loss: 0.3653 - acc

In [21]:

test1 = pd.DataFrame(scaler.transform(test_color_x), columns=test_color_x.columns, index=test_color_x.index)

In [22]:
pred_mat = model.predict(test1)

In [23]:
pred_mat

array([[5.4866156e-14, 5.2194810e-07, 3.4550685e-05, ..., 2.6785806e-08,
        2.3176090e-05, 3.2420689e-06],
       [4.1028168e-05, 3.3701617e-06, 1.0900740e-09, ..., 1.2810546e-09,
        1.9092156e-05, 5.4314086e-04],
       [3.9668307e-06, 5.1303942e-07, 3.9750504e-20, ..., 3.7887085e-06,
        1.6173497e-06, 5.2409177e-08],
       ...,
       [1.1279975e-04, 1.5110738e-04, 1.1076196e-12, ..., 1.5231346e-13,
        2.2826967e-05, 6.7254834e-02],
       [1.4781172e-09, 9.2716568e-13, 8.0528927e-14, ..., 3.1450019e-23,
        7.6489757e-09, 5.7735661e-04],
       [5.8410849e-05, 3.1726231e-06, 2.5909770e-20, ..., 4.1048955e-14,
        2.9478587e-08, 4.4994213e-06]], dtype=float32)

In [24]:
submission = pd.DataFrame(data=pred_mat, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('./data/dacon_g/submission_DL_2_1.csv', index=True)